我们这里给四缺陷打上伪标签

In [1]:
import sys
sys.path.append('..')

In [2]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from py_file.func_Test import Test_Func
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch

In [3]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

In [4]:
# 定义训练的设备
if torch.cuda.is_available():
    device = torch.device('cuda:0') # 只有一张显卡的话，'cuda'和'cuda:0'是一样的
else:
    device = torch.device('cpu')
print(f'使用的设备为：{device}')

使用的设备为：cuda:0


# 加载数据和模型

## 获取数据

In [5]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)
print(train_data.files)

['original_wm', 'label_one_hot', 'denoise_wm', 'label_name']


In [6]:
ori_wm = train_data['original_wm']
wm = train_data['denoise_wm']
label = train_data['label_name']
label_oh = train_data['label_one_hot']
mul_ori_wm = []
mul_wm = []
mul_label = []
mul_label_oh = []
for i in range(len(label_oh)):
    if label_oh[i].sum() >= 2:
        mul_ori_wm.append(np.array(ori_wm[i]))
        mul_wm.append(np.array(wm[i]))  # 元素是tensor的列表无法转换为张量，所以这里把列表里的元素转换为np.array
        mul_label.append(label[i])
        mul_label_oh.append(label_oh[i])

In [7]:
mul_wm_tensor = trans(torch.tensor(mul_wm, dtype=torch.float32))
mul_wm_tensor = torch.reshape(mul_wm_tensor,(-1,1,224,224))  # 给图片加上通道
print(mul_wm_tensor.shape)

/tmp/ipykernel_20716/1119019278.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  mul_wm_tensor = trans(torch.tensor(mul_wm, dtype=torch.float32))


torch.Size([20300, 1, 224, 224])


## 获取类别名称和多热编码对应的字典

In [8]:
mul_label_unique, label_indices = np.unique(mul_label, return_index=True)
print(mul_label_unique)
print(label_indices)
# print(two_label_unique[indices])
mul_label_oh_unique ,label_oh_indices = np.unique(mul_label_oh,axis=0, return_index=True)
print(mul_label_oh_unique)
print(label_oh_indices)

['C+EL' 'C+EL+L' 'C+EL+L+S' 'C+EL+S' 'C+ER' 'C+ER+L' 'C+ER+L+S' 'C+ER+S'
 'C+L' 'C+L+S' 'C+S' 'D+EL' 'D+EL+L' 'D+EL+L+S' 'D+EL+S' 'D+ER' 'D+ER+L'
 'D+ER+L+S' 'D+ER+S' 'D+L' 'D+L+S' 'D+S' 'EL+L' 'EL+L+S' 'EL+S' 'ER+L'
 'ER+L+S' 'ER+S' 'L+S']
[  700  3500  2800     0  2100  4900  4200  1400  6300  5600  7000  8400
 11200 10500  7700  9800 12600 11900  9100 14000 13300 14700 16100 15400
 18900 17500 16800 19600 18200]
[[0 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 1 1 0 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 1 0 1 0 0 0]
 [0 0 1 0 1 0 1 0]
 [0 1 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 0]
 [0 1 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 0]
 [0 1 0 1 0 0 1 0]
 [0 1 0 1 1 0 0 0]
 [0 1 0 1 1 0 1 0]
 [0 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 1 0]
 [0 1 1 0 1 0 0 0]
 [0 1 1 0 1 0 1 0]
 [1 0 0 0 0 0 1 0]
 [1 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 1 0]
 [1 0 0 1 0 0 0 0]
 [1 0 0 1 0 0 1 0]
 [1 0 0 1 1 0 0 0]
 [1 0 0 1 1 0 1 0]
 [1 0 1 0 0 0 0 0]
 [1 0 1 0 0 0 1 0]
 [1 0 1 0 1 0 0 0]
 [1 0 1 0 1 0 1 0]]
[18200 19600 17500 16800 1890

In [9]:
oh_dict = {}
for index in label_indices:
    oh_dict[mul_label[index]] = mul_label_oh[index]
print(oh_dict)

{'C+EL': array([1, 0, 1, 0, 0, 0, 0, 0], dtype=int32), 'C+EL+L': array([1, 0, 1, 0, 1, 0, 0, 0], dtype=int32), 'C+EL+L+S': array([1, 0, 1, 0, 1, 0, 1, 0], dtype=int32), 'C+EL+S': array([1, 0, 1, 0, 0, 0, 1, 0], dtype=int32), 'C+ER': array([1, 0, 0, 1, 0, 0, 0, 0], dtype=int32), 'C+ER+L': array([1, 0, 0, 1, 1, 0, 0, 0], dtype=int32), 'C+ER+L+S': array([1, 0, 0, 1, 1, 0, 1, 0], dtype=int32), 'C+ER+S': array([1, 0, 0, 1, 0, 0, 1, 0], dtype=int32), 'C+L': array([1, 0, 0, 0, 1, 0, 0, 0], dtype=int32), 'C+L+S': array([1, 0, 0, 0, 1, 0, 1, 0], dtype=int32), 'C+S': array([1, 0, 0, 0, 0, 0, 1, 0], dtype=int32), 'D+EL': array([0, 1, 1, 0, 0, 0, 0, 0], dtype=int32), 'D+EL+L': array([0, 1, 1, 0, 1, 0, 0, 0], dtype=int32), 'D+EL+L+S': array([0, 1, 1, 0, 1, 0, 1, 0], dtype=int32), 'D+EL+S': array([0, 1, 1, 0, 0, 0, 1, 0], dtype=int32), 'D+ER': array([0, 1, 0, 1, 0, 0, 0, 0], dtype=int32), 'D+ER+L': array([0, 1, 0, 1, 1, 0, 0, 0], dtype=int32), 'D+ER+L+S': array([0, 1, 0, 1, 1, 0, 1, 0], dtype=int32)

## 加载模型和语义

In [10]:
import pickle
model = torch.load('model_saved_pseudo/train_single_two_three.pth')
model.eval()

with open('updated_semantic_1_2_3_4/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3_4/updated_mul_dict.pkl', 'rb') as file:
    mul_defect_att = pickle.load(file)

from collections import Counter
# 加载二、三、四缺陷的字典
two_defect_att = {}
three_defect_att = {}
four_defect_att = {}
for label in mul_defect_att.keys():
    count = Counter(label)
    if count['+'] == 1:
        two_defect_att[label] = mul_defect_att[label]
    if count['+'] == 2:
        three_defect_att[label] = mul_defect_att[label]
    if count['+'] == 3:
        four_defect_att[label] = mul_defect_att[label]

In [11]:
mul_dataset = MyDataSet(mul_wm_tensor, mul_label)
mul_loader = DataLoader(mul_dataset, batch_size=32, shuffle=False)

# 定义预测标签的函数

In [12]:
def cosine_similarity(v1, v2):  # 参数v1,v2是np.array,不能是tensor，可以用np.array()将tensor转换为array
    # 计算两个向量的点积
    dot_product = np.dot(v1, v2)
    # 计算两个向量的模
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    # 计算余弦相似度
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

In [13]:
# 给一个属性张量判断标签
def max_similarity(att_tensor, defe_dict, t = -1):  # 使用余弦相似度
    max_sim = -2
    att_array = np.array(att_tensor.cpu())  # 在GPU上的张量无法转换为array，只有在CPU上的张量才能转换为array
    for label,attribute in defe_dict.items():
        attribute_array = np.array(attribute)
        simi = cosine_similarity(att_array,attribute_array)

        if simi>max_sim:
            max_sim = simi
            predict_label = label
    
    if t!=-1 and max_sim<t:
        predict_label = 'unknown'
    
    return predict_label  # 这是一个str类型

In [14]:
# 从一个批次的输出属性中，获取预测的标签
def Predict_label(outputs, defe_att_dic, t = -1):
        outputs_labels = []   # tensor只能将元素为数字类型的列表转换为tensor
        for out in outputs:
            outputs_labels.append(max_similarity(out,defe_att_dic,t))
        return outputs_labels


# 从dataloader中提取label
def get_label(model, dataloader, att_dic, t = -1):
    label_true = []
    label_pre = []
    with torch.no_grad(): 
        for data in dataloader:  
            imgs,labels = data

            for label in labels:
                label_true.append(label)

            imgs = imgs.to(device)
            
            outputs = model(imgs)

            y_pre = Predict_label(outputs, att_dic, t)
            label_pre = label_pre + y_pre

    return label_true,label_pre

In [15]:
def clean_data(ori_wm, denoised_wm, real_label, pred_label):
    cleaned_ori_wm = []
    cleaned_denoised_wm = []
    cleaned_real_label = []
    cleaned_pred_label = []

    for i in range(len(pred_label)):
        if pred_label[i] != 'unknown':
            cleaned_ori_wm.append(ori_wm[i])
            cleaned_denoised_wm.append(denoised_wm[i])
            cleaned_real_label.append(real_label[i])
            cleaned_pred_label.append(pred_label[i])
    
    return cleaned_ori_wm, cleaned_denoised_wm, cleaned_real_label, cleaned_pred_label

In [16]:
# 获取伪标签的粗粒度准确率和细粒度准确率
def get_acc(real_label, pred_label, label_unique):
    Coarse_grained_num = 0
    for it in real_label:
        if it in label_unique:
            Coarse_grained_num += 1
    Coarse_grained_acc = Coarse_grained_num / len(real_label)

    real_label_array = np.array(real_label)
    pred_label_array = np.array(pred_label)
    Fine_grained_acc = np.sum(real_label_array == pred_label_array) / len(real_label)

    return Coarse_grained_acc, Fine_grained_acc, len(real_label)

# 获取被判断为四故障的样本

In [17]:
mul_real_label, mul_predict_label = get_label(model, mul_loader, mul_defect_att)

In [18]:
four_label_unique = four_defect_att.keys()
print(len(four_label_unique))
print(four_label_unique)

4
dict_keys(['C+EL+L+S', 'C+ER+L+S', 'D+EL+L+S', 'D+ER+L+S'])


In [19]:
predicted_four_wm = []
predicted_four_ori_wm = []
predicted_four_real_label = []
for i in range(len(mul_predict_label)):
    if mul_predict_label[i] in four_label_unique:
        predicted_four_wm.append(mul_wm[i])
        predicted_four_ori_wm.append(mul_ori_wm[i])
        predicted_four_real_label.append(mul_label[i])

In [20]:
n = 0
for i in range(len(predicted_four_wm)):
    if predicted_four_real_label[i] not in four_label_unique:
        n = n + 1
print(n, len(predicted_four_wm))
print(n/len(predicted_four_wm))

372 1673
0.22235505080693366


In [21]:
predicted_four_wm_tensor = trans(torch.tensor(np.array(predicted_four_wm), dtype=torch.float32))
predicted_four_wm_tensor = torch.reshape(predicted_four_wm_tensor,(-1,1,224,224))

predicted_four_real_label = np.array(predicted_four_real_label)
print(predicted_four_wm_tensor.shape, predicted_four_real_label.shape)

torch.Size([1673, 1, 224, 224]) (1673,)


In [22]:
predicted_four_dataset = MyDataSet(predicted_four_wm_tensor, predicted_four_real_label)
predicted_four_loader = DataLoader(predicted_four_dataset, batch_size=32, shuffle=False)

# 试验参数T

In [57]:
T = 0.980

In [58]:
real_label, predict_label = get_label(model, predicted_four_loader, four_defect_att, T)
saved_ori_wm, saved_wm, saved_real_label, saved_predict_label = clean_data(predicted_four_ori_wm, predicted_four_wm,
 real_label, predict_label)

label_unique = list(four_defect_att.keys())
coarse_acc, fine_acc, num = get_acc(saved_real_label, saved_predict_label, label_unique)

print(f"coarse_acc: {coarse_acc}, fine_acc: {fine_acc}, num: {num}")

coarse_acc: 0.7820748925721301, fine_acc: 0.583793738489871, num: 1629


# 保存打了四故障伪标签的样本

In [59]:
saved_label_oh = []
for l in saved_predict_label:
    saved_label_oh.append(oh_dict[l])
    
saved_label_oh = np.array(saved_label_oh)
print(saved_label_oh.shape)

(1629, 8)


In [60]:
saved_ori_wm = np.array(saved_ori_wm)
saved_wm = np.array(saved_wm)
saved_predict_label = np.array(saved_predict_label)
print(saved_ori_wm.shape)
print(saved_wm.shape)
print(saved_predict_label.shape)

(1629, 52, 52)
(1629, 52, 52)
(1629,)


In [61]:
np.savez('data_fake_label/four_fake_label_WM.npz', original_wm = saved_ori_wm, label_one_hot = saved_label_oh,
 denoise_wm = saved_wm, label_name = saved_predict_label)